In [2]:
import IPython
import pandas as pd
from utils import *
from calculate_joint_angles import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data_path = "../data/small_data/pose_saved.csv"
data = pd.read_csv(data_path)
print(data.dtypes)
data.head()

rgb_frameid     int64
kp3ds          object
kp2ds          object
bboxes         object
dtype: object


,rgb_frameid,kp3ds,kp2ds,bboxes
0,5000,"[[2433.527603034382, -1938.5729978558697, 65.7...","[[[1008.9975321765137, 101.44294880551615, 0.7...","[[743.9833984375, 0.0, 1234.8826904296875, 622..."
1,5001,"[[2446.607236443355, -1934.9092386957425, 67.5...","[[[999.6039120528574, 99.06533975351539, 0.758...","[[740.1781616210938, 0.0, 1231.525390625, 622...."
2,5002,"[[2460.826163175043, -1930.3790846162383, 69.3...","[[[992.6256386363831, 97.69814209614634, 0.727...","[[736.6728515625, 0.089996337890625, 1228.7199..."
3,5003,"[[2473.6984260388895, -1926.4497944254015, 73....","[[[985.1741546289431, 95.75803250340277, 0.752...","[[734.6681518554688, 0.10302734375, 1223.74499..."
4,5004,"[[2489.1893336653734, -1925.6113847420506, 73....","[[[978.2528523690814, 95.23409782663788, 0.765...","[[733.6686401367188, 0.173248291015625, 1216.8..."


### Let's only take the first two columns


In [4]:
df = data[['rgb_frameid', 'kp3ds']]
df.head()

,rgb_frameid,kp3ds
0,5000,"[[2433.527603034382, -1938.5729978558697, 65.7..."
1,5001,"[[2446.607236443355, -1934.9092386957425, 67.5..."
2,5002,"[[2460.826163175043, -1930.3790846162383, 69.3..."
3,5003,"[[2473.6984260388895, -1926.4497944254015, 73...."
4,5004,"[[2489.1893336653734, -1925.6113847420506, 73...."


In [5]:
print(type(df.kp3ds[0]))
df.kp3ds[0]

<class 'str'>


'[[2433.527603034382, -1938.5729978558697, 65.77853319876616, 0.715919840335846], [2455.1936794872177, -1931.8907779942238, 97.20523561103043, 0.6419889330863953], [2413.1285141606477, -1915.3594788777505, 100.405452670196, 0.6129101574420929], [2553.3050698424627, -1851.7950717623419, 114.90635783137866, 0.7434812635183334], [2406.47320246087, -1791.7937153377102, 130.30197906455479, 0.58548554033041], [2645.352619537392, -1677.0320842443841, -34.92217267652428, 0.721489417552948], [2305.953153691892, -1832.9165705828136, -22.7491032373318, 0.675788402557373], [2713.5425084514063, -1596.1945567660148, -283.31397363860276, 0.6877377976973852], [2205.742184616739, -2043.9587716844408, -78.7145570247363, 0.6339661717414856], [2746.769581107897, -1777.5664535076626, -405.71990057299956, 0.6417463049292564], [2196.6960493973734, -2288.6940364719085, -26.30439482351236, 0.7618403911590577], [2601.681535535075, -1730.8660615202673, -554.1895844242209, 0.642460686819894], [2415.360061812779, 

### Let's convert the string into vector

In [6]:
df['kp3ds'] = df['kp3ds'].apply(str2float)
df['kp3ds']

/var/folders/_q/5gxcvv9j1dg_pzh17qyfbkk40000gn/T/ipykernel_71780/1860301907.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kp3ds'] = df['kp3ds'].apply(str2float)


0       [[2433.527603034382, -1938.5729978558697, 65.7...
1       [[2446.607236443355, -1934.9092386957425, 67.5...
2       [[2460.826163175043, -1930.3790846162383, 69.3...
3       [[2473.6984260388895, -1926.4497944254015, 73....
4       [[2489.1893336653734, -1925.6113847420506, 73....
                              ...                        
4995    [[2783.777098030839, -480.39212498854664, -45....
4996    [[2786.0874771535578, -479.94158441571295, -46...
4997    [[2791.0247204375196, -479.5124704274368, -46....
4998    [[2793.123929938673, -480.28451711442835, -46....
4999    [[2799.4689841766367, -479.83214699544794, -46...
Name: kp3ds, Length: 5000, dtype: object

### Let's separate each kp3ds line into columns for each body points, and remove kp3ds column

In [7]:
df_points = get_df_points(df)
df_points.head(2)

,point_0,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,point_10,point_11,point_12,point_13,point_14,point_15,point_16
0,"[2433.527603034382, -1938.5729978558697, 65.77...","[2455.1936794872177, -1931.8907779942238, 97.2...","[2413.1285141606477, -1915.3594788777505, 100....","[2553.3050698424627, -1851.7950717623419, 114....","[2406.47320246087, -1791.7937153377102, 130.30...","[2645.352619537392, -1677.0320842443841, -34.9...","[2305.953153691892, -1832.9165705828136, -22.7...","[2713.5425084514063, -1596.1945567660148, -283...","[2205.742184616739, -2043.9587716844408, -78.7...","[2746.769581107897, -1777.5664535076626, -405....","[2196.6960493973734, -2288.6940364719085, -26....","[2601.681535535075, -1730.8660615202673, -554....","[2415.360061812779, -1740.6648141592275, -579....","[2679.9952724415434, -1787.690377703567, -936....","[2400.8627281541926, -1778.5203763504858, -100...","[2756.2899011781155, -1799.6261441022816, -135...","[2414.4655146949895, -1684.1135292174695, -136..."
1,"[2446.607236443355, -1934.9092386957425, 67.53...","[2467.2217900211476, -1930.3347251724365, 97.9...","[2428.116065842915, -1907.5241451478714, 101.8...","[2569.8145999447324, -1852.6092353611177, 115....","[2422.253575661983, -1782.1572547434125, 129.7...","[2665.6696670464685, -1679.922232055294, -38.2...","[2318.2960842243237, -1820.4548819188512, -18....","[2725.426330672456, -1602.0102743236503, -296....","[2208.9420826541023, -2023.5744809148555, -71....","[2750.7631518119424, -1794.6567370184462, -407...","[2201.630793546887, -2268.631210247707, -14.05...","[2610.394828575268, -1748.6124215080272, -551....","[2419.367793916796, -1747.9863605084934, -570....","[2687.5016372386776, -1798.4410208952943, -933...","[2400.7339061752123, -1788.0900152347378, -100...","[2760.4205598737426, -1799.6442142374067, -135...","[2415.447631110754, -1698.3153128577876, -1366..."


In [8]:
df = pd.concat([df, df_points], axis=1)
df.drop(columns=['kp3ds'], inplace=True)
df.head(3)


,rgb_frameid,point_0,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,point_10,point_11,point_12,point_13,point_14,point_15,point_16
0,5000,"[2433.527603034382, -1938.5729978558697, 65.77...","[2455.1936794872177, -1931.8907779942238, 97.2...","[2413.1285141606477, -1915.3594788777505, 100....","[2553.3050698424627, -1851.7950717623419, 114....","[2406.47320246087, -1791.7937153377102, 130.30...","[2645.352619537392, -1677.0320842443841, -34.9...","[2305.953153691892, -1832.9165705828136, -22.7...","[2713.5425084514063, -1596.1945567660148, -283...","[2205.742184616739, -2043.9587716844408, -78.7...","[2746.769581107897, -1777.5664535076626, -405....","[2196.6960493973734, -2288.6940364719085, -26....","[2601.681535535075, -1730.8660615202673, -554....","[2415.360061812779, -1740.6648141592275, -579....","[2679.9952724415434, -1787.690377703567, -936....","[2400.8627281541926, -1778.5203763504858, -100...","[2756.2899011781155, -1799.6261441022816, -135...","[2414.4655146949895, -1684.1135292174695, -136..."
1,5001,"[2446.607236443355, -1934.9092386957425, 67.53...","[2467.2217900211476, -1930.3347251724365, 97.9...","[2428.116065842915, -1907.5241451478714, 101.8...","[2569.8145999447324, -1852.6092353611177, 115....","[2422.253575661983, -1782.1572547434125, 129.7...","[2665.6696670464685, -1679.922232055294, -38.2...","[2318.2960842243237, -1820.4548819188512, -18....","[2725.426330672456, -1602.0102743236503, -296....","[2208.9420826541023, -2023.5744809148555, -71....","[2750.7631518119424, -1794.6567370184462, -407...","[2201.630793546887, -2268.631210247707, -14.05...","[2610.394828575268, -1748.6124215080272, -551....","[2419.367793916796, -1747.9863605084934, -570....","[2687.5016372386776, -1798.4410208952943, -933...","[2400.7339061752123, -1788.0900152347378, -100...","[2760.4205598737426, -1799.6442142374067, -135...","[2415.447631110754, -1698.3153128577876, -1366..."
2,5002,"[2460.826163175043, -1930.3790846162383, 69.34...","[2484.918090524629, -1924.8040044837676, 99.79...","[2440.8411861266345, -1900.254950299179, 104.3...","[2584.0739331874606, -1851.960169894501, 113.9...","[2435.4652524619855, -1776.6510228370694, 132....","[2684.776549436161, -1687.3026095059545, -43.6...","[2329.3454519494003, -1808.3264969982044, -19....","[2743.056079751839, -1614.688905862936, -304.6...","[2195.1750290174386, -2006.137971398989, -65.8...","[2755.032244545425, -1812.6779647475068, -409....","[2206.279819898573, -2244.3941945026313, -3.28...","[2619.789612932464, -1751.2821608695785, -572....","[2427.258121931534, -1746.360628016013, -587.8...","[2695.004890749521, -1794.4201805342605, -942....","[2404.6687226515774, -1789.2744484614768, -100...","[2769.543079408299, -1809.2850525834021, -1367...","[2415.272491615325, -1700.7187023871097, -1364..."


#### The framerate is 30 FPS, let's convert the frameid column into time

In [9]:
df['rgb_frameid'] = (df['rgb_frameid'] - 5000)/30
df.rename(columns={'rgb_frameid': 'time'}, inplace=True)
df.head(2)

,time,point_0,point_1,point_2,point_3,point_4,point_5,point_6,point_7,point_8,point_9,point_10,point_11,point_12,point_13,point_14,point_15,point_16
0,0.000000,"[2433.527603034382, -1938.5729978558697, 65.77...","[2455.1936794872177, -1931.8907779942238, 97.2...","[2413.1285141606477, -1915.3594788777505, 100....","[2553.3050698424627, -1851.7950717623419, 114....","[2406.47320246087, -1791.7937153377102, 130.30...","[2645.352619537392, -1677.0320842443841, -34.9...","[2305.953153691892, -1832.9165705828136, -22.7...","[2713.5425084514063, -1596.1945567660148, -283...","[2205.742184616739, -2043.9587716844408, -78.7...","[2746.769581107897, -1777.5664535076626, -405....","[2196.6960493973734, -2288.6940364719085, -26....","[2601.681535535075, -1730.8660615202673, -554....","[2415.360061812779, -1740.6648141592275, -579....","[2679.9952724415434, -1787.690377703567, -936....","[2400.8627281541926, -1778.5203763504858, -100...","[2756.2899011781155, -1799.6261441022816, -135...","[2414.4655146949895, -1684.1135292174695, -136..."
1,0.033333,"[2446.607236443355, -1934.9092386957425, 67.53...","[2467.2217900211476, -1930.3347251724365, 97.9...","[2428.116065842915, -1907.5241451478714, 101.8...","[2569.8145999447324, -1852.6092353611177, 115....","[2422.253575661983, -1782.1572547434125, 129.7...","[2665.6696670464685, -1679.922232055294, -38.2...","[2318.2960842243237, -1820.4548819188512, -18....","[2725.426330672456, -1602.0102743236503, -296....","[2208.9420826541023, -2023.5744809148555, -71....","[2750.7631518119424, -1794.6567370184462, -407...","[2201.630793546887, -2268.631210247707, -14.05...","[2610.394828575268, -1748.6124215080272, -551....","[2419.367793916796, -1747.9863605084934, -570....","[2687.5016372386776, -1798.4410208952943, -933...","[2400.7339061752123, -1788.0900152347378, -100...","[2760.4205598737426, -1799.6442142374067, -135...","[2415.447631110754, -1698.3153128577876, -1366..."


### Placement of 3D body points
<a href=https://github.com/open-mmlab/mmpose/tree/main/projects/rtmpose><img src="https://user-images.githubusercontent.com/13503330/245036409-2417e4f7-2203-468f-bad0-e7a6a6bf8251.jpg"  width="150" height="300">

![alt text](img/3d_body_points_axes.jpeg "Title")

#### Which one is left/right ?
At the beginning of the video, the participants right foot has the lowest x value. By taking a look at points 15 and 16 x values, the **point 16** corresponds to the **right** foot.

From this information, we can better precise the column names in our dataset.

In [10]:
df.rename(columns = {'point_5':'shoulder_l', 'point_6':'shoulder_r', 
                     'point_8':'elbow_r', 'point_7':'elbow_l', 
                     'point_10':'wrist_r', 'point_9':'wrist_l',
                     'point_12':'hip_r', 'point_11':'hip_l'}, inplace = True)
df.head(2)

,time,point_0,point_1,point_2,point_3,point_4,shoulder_l,shoulder_r,elbow_l,elbow_r,wrist_l,wrist_r,hip_l,hip_r,point_13,point_14,point_15,point_16
0,0.000000,"[2433.527603034382, -1938.5729978558697, 65.77...","[2455.1936794872177, -1931.8907779942238, 97.2...","[2413.1285141606477, -1915.3594788777505, 100....","[2553.3050698424627, -1851.7950717623419, 114....","[2406.47320246087, -1791.7937153377102, 130.30...","[2645.352619537392, -1677.0320842443841, -34.9...","[2305.953153691892, -1832.9165705828136, -22.7...","[2713.5425084514063, -1596.1945567660148, -283...","[2205.742184616739, -2043.9587716844408, -78.7...","[2746.769581107897, -1777.5664535076626, -405....","[2196.6960493973734, -2288.6940364719085, -26....","[2601.681535535075, -1730.8660615202673, -554....","[2415.360061812779, -1740.6648141592275, -579....","[2679.9952724415434, -1787.690377703567, -936....","[2400.8627281541926, -1778.5203763504858, -100...","[2756.2899011781155, -1799.6261441022816, -135...","[2414.4655146949895, -1684.1135292174695, -136..."
1,0.033333,"[2446.607236443355, -1934.9092386957425, 67.53...","[2467.2217900211476, -1930.3347251724365, 97.9...","[2428.116065842915, -1907.5241451478714, 101.8...","[2569.8145999447324, -1852.6092353611177, 115....","[2422.253575661983, -1782.1572547434125, 129.7...","[2665.6696670464685, -1679.922232055294, -38.2...","[2318.2960842243237, -1820.4548819188512, -18....","[2725.426330672456, -1602.0102743236503, -296....","[2208.9420826541023, -2023.5744809148555, -71....","[2750.7631518119424, -1794.6567370184462, -407...","[2201.630793546887, -2268.631210247707, -14.05...","[2610.394828575268, -1748.6124215080272, -551....","[2419.367793916796, -1747.9863605084934, -570....","[2687.5016372386776, -1798.4410208952943, -933...","[2400.7339061752123, -1788.0900152347378, -100...","[2760.4205598737426, -1799.6442142374067, -135...","[2415.447631110754, -1698.3153128577876, -1366..."


## Intermediary features

### Get angles 

In [11]:
a = df.shoulder_l[0]
b = df.elbow_l[0]
c = df.wrist_l[0]
print(angle(a, b, c))

107.54186179726136


### Get angle velocities

In [12]:
#Participant is picking a knife between 6-7s
period = time_id(df, 6, 7)
period


[180, 210]

## Final features

### Get joint angles

At first, I just tried to get the angles between 2 vectors (for example shoulder-elbow and wrist-elbow) to define it as the "angle joint". But then I realized that one can not just do that, since it will be then not sufficient to define the flexion, ab/adduction etc. 

I went back to read some literature on how to define joint angles based on 3D body points, and found some interesting references :
- [Method to estimate joint angles + code](https://temugeb.github.io/python/motion_capture/2021/09/16/joint_rotations.html)
It explains how to find joint angles based on basic body pose (T-pose)
- [This paper](https://www.nature.com/articles/s41591-022-02045-1#Sec9) explains that joint angles can be extracted in 3 d.o.f for each joint, by following the International Society of Biomechanics recommendations for Euler angle extractions of X (abduction/adduction), Y (internal/external rotation) and Z (flexion/extension).
- [This paper](https://www.rachelvitali.com/docs/Determining%20anatomical%20frames%20via%20inertial%20motion%20capture%20-%20a%20survey%20of%20methods.pdf) says that there are different methods used to define anatomical frames of reference for inertial motion capture.

--> So I should come up myself with a method to define the anatomical frame of reference.

**Here is my method to try to express the joint angles in every anatomical plane :**

1. Find a way to define body coordinates in order to easily select the anatomical planes (try reproduce the body coordinates as shown below in the picture)
2. Find a way to create the rotation matric that rotates the usual coordinate system to the body coordinates
3. Find a way to express the 3D point into body coordinates s.t. one can easily select the plane (for ex. Z-X plane for sagittal plane) to then compute the angle

<a href = "https://en.wikipedia.org/wiki/Anatomical_plane"><img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/2/24/Human_anatomy_planes%2C_labeled.svg/1200px-Human_anatomy_planes%2C_labeled.svg.png" width = 200/></a>

#### 1. Body coordinates system definition
Origin is at the middle of the hip joints

Z-axis : From origin to middle of shoulder joints

Y-axis : From origin to right hip joint

X-axis : Perpendicular to Z and Y, in head direction (point_0)
--> Take point_0, put same z coordinate as middle of hip

In [13]:
body_df = pd.DataFrame()
body_df['origin'] = (df['hip_l'] + df['hip_r'])/2

#Direction of z-axis
body_df['z'] = (df['shoulder_l'] + df['shoulder_r'])/2 - body_df['origin']

#Direction of y-axis 
body_df['y'] = df['hip_r'] - body_df['origin']

#Direction of x-axis 
body_df['head'] = df['point_0']

In [31]:
#We don't need the points around the head to calculate the joint angles
new_df = np.array(df.drop(columns=['time', 'point_0', 'point_1', 'point_2', 'point_3', 'point_4']))

In [35]:
#Code from elsewhere

#rotate to orient the pose better
R = utils.get_R_z(np.pi/2)
# number of frames
for framenum in range(new_df.shape[0]):
    #number of points
    for pt_num in range(new_df.shape[1]):
        new_df[framenum, pt_num] = R @ new_df[framenum, pt_num]

kpts = convert_to_dictionary(new_df)
"""
add_hips_and_neck(kpts)
filtered_kpts = median_filter(kpts)
get_bone_lengths(filtered_kpts)
get_base_skeleton(filtered_kpts)

calculate_joint_angles(filtered_kpts)
#draw_skeleton_from_joint_coordinates(filtered_kpts)
draw_skeleton_from_joint_angles(filtered_kpts)
"""

{'lefthip': array([array([-2601.68153554,  1730.86606152,  -554.18958442]),
       array([-2610.39482858,  1748.61242151,  -551.45182012]),
       array([-2619.78961293,  1751.28216087,  -572.70058926]), ...,
       array([-2722.47602567,   731.08111904,  -564.94150451]),
       array([-2724.2139364 ,   731.51111298,  -566.69489634]),
       array([-2724.53034652,   734.17494493,  -566.86376761])],
      dtype=object), 'leftknee': array([array([-2679.99527244,  1787.6903777 ,  -936.18973483]),
       array([-2687.50163724,  1798.4410209 ,  -933.06871916]),
       array([-2695.00489075,  1794.42018053,  -942.18910421]), ...,
       array([-2752.17501747,   733.3186246 ,  -973.84042009]),
       array([-2753.78925114,   731.90346839,  -975.74804636]),
       array([-2753.02276536,   731.26780962,  -974.271832  ])],
      dtype=object), 'leftfoot': array([array([-2756.28990118,  1799.6261441 , -1353.46554887]),
       array([-2760.42055987,  1799.64421424, -1352.8391815 ]),
       array([

'\nadd_hips_and_neck(kpts)\nfiltered_kpts = median_filter(kpts)\nget_bone_lengths(filtered_kpts)\nget_base_skeleton(filtered_kpts)\n\ncalculate_joint_angles(filtered_kpts)\n#draw_skeleton_from_joint_coordinates(filtered_kpts)\ndraw_skeleton_from_joint_angles(filtered_kpts)\n'

#### 2.1 Shoulder flexion/extension
The angle is computed between the vector shoulder-elbow and shoulder-hip on sagittal plane.

<a href="https://physicaltherapyweb.com/hyper-extension-internal-rotation-heri-test/"><img src ="https://i0.wp.com/physicaltherapyweb.com/wp-content/uploads/2016/08/shoulder-flexion-and-extension.png?fit=329%2C410&ssl=1" width = 200/></a>





#### 2.2 Shoulder ab/adduction

#### 3. Elbow flexion/extension

From litterature, the elbow joint allows us to perform flexion–extension and pronation–supination movement.

<a href="https://www.researchgate.net/figure/Normal-range-of-motion-for-the-elbow-joint_fig2_319499390"><img src="https://www.researchgate.net/publication/319499390/figure/fig2/AS:1086484513857552@1636049502528/Normal-range-of-motion-for-the-elbow-joint.jpg" alt="Normal range of motion for the elbow joint." width = "300" height = "300"/></a>

But since in our dataset, the pronation-supination movement is not captured, the joint angle of the elbow simply corresponds to the flexion-extension movement, meaning : 
- Points 5 - 7 - 9 for the left arm
- Points 6 - 8 - 10 for the right arm

ref : Zwerus, E. L., Willigenburg, N. W., Scholtes, V. A., Somford, M. P., Eygendaal, D., & van den Bekerom, M. P. (2019). Normative values and affecting factors for the elbow range of motion. Shoulder & elbow, 11(3), 215–224. https://doi.org/10.1177/1758573217728711


In [14]:
a = df.shoulder_l[0]
b = df.elbow_l[0]
c = df.wrist_l[0]
print(angle(a, b, c))

107.54186179726136


#### 4.1 Trunk flexion

#### 4.2 Trunk rotation